# 

In [2]:
import tensorflow as tf

2025-10-02 09:32:08.237741: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-02 09:32:08.273115: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-02 09:32:09.135135: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
class LeNet(tf.keras.Model):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=6, kernel_size=5,
                                            activation='sigmoid', padding='same')
        self.pool1 = tf.keras.layers.AvgPool2D(pool_size=2, strides=2)
        self.conv2 = tf.keras.layers.Conv2D(filters=16, kernel_size=5,
                                            activation='sigmoid')
        self.pool2 = tf.keras.layers.AvgPool2D(pool_size=2, strides=2)
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(120, activation='sigmoid')
        self.dense2 = tf.keras.layers.Dense(84, activation='sigmoid')
        self.dense3 = tf.keras.layers.Dense(num_classes)

    def call(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        return x

In [7]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# adding a channel dimension to the input
x_train = tf.expand_dims(x_train, axis=-1) 
x_test = tf.expand_dims(x_test, axis=-1)

y_train = tf.one_hot(y_train, depth=10)
y_test = tf.one_hot(y_test, depth=10)

batch_size = 128
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.batch(batch_size)

print("Training dataset shape:", tf.data.Dataset.cardinality(train_dataset).numpy())
print("Testing dataset shape:", tf.data.Dataset.cardinality(test_dataset).numpy())

Training dataset shape: 469
Testing dataset shape: 79


I0000 00:00:1759392269.691071    6453 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2133 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [8]:
model = LeNet(num_classes=10)
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
epochs = 10

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    total_loss = 0.0
    num_batches = 0
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)
            loss_value = loss_fn(y_batch_train, logits)

        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        total_loss += loss_value.numpy()
        num_batches += 1

    avg_loss = total_loss / num_batches
    print(f"  Training Loss: {avg_loss:.4f}")

Epoch 1/10


2025-10-02 10:13:49.359406: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91301
2025-10-02 10:13:56.468942: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


  Training Loss: 2.3084
Epoch 2/10


2025-10-02 10:14:02.880074: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


  Training Loss: 2.3048
Epoch 3/10
  Training Loss: 2.2667
Epoch 4/10


2025-10-02 10:14:14.900268: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


  Training Loss: 1.5117
Epoch 5/10
  Training Loss: 1.0719
Epoch 6/10
  Training Loss: 0.9308
Epoch 7/10
  Training Loss: 0.8440
Epoch 8/10


2025-10-02 10:14:39.061044: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


  Training Loss: 0.7869
Epoch 9/10
  Training Loss: 0.7385
Epoch 10/10
  Training Loss: 0.7011


In [9]:
correct_predictions = 0
total_samples = 0

for x_batch_test, y_batch_test in test_dataset:
    logits = model(x_batch_test, training=False)
    predicted_classes = tf.argmax(logits, axis=1)
    true_classes = tf.argmax(y_batch_test, axis=1)
    correct_predictions += tf.reduce_sum(tf.cast(tf.equal(predicted_classes, true_classes), dtype=tf.int32)).numpy()
    total_samples += x_batch_test.shape[0]

accuracy = correct_predictions / total_samples
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.7369
